In [1]:
import holmes.experiment as Ex
import tempfile
import pprint
import json
import os
from terrarium.flatten import flatten_dict, expand_dict
import tempfile

In [ ]:
temp_dir_obj = tempfile.TemporaryDirectory()
temp_dir = temp_dir_obj.name
package_location = Ex.staging.prep_holmes_package(
    staging_path=temp_dir, 
    commit_hash='ca6ccd63f270d603ffa9bdf5a7ced184d6339796'
)

detects_dict={}
for video_data in Ex.val_data:
    detects_dict[video_data['video_id']] = video_data['video_path'].replace('processed','detections')

def_conf_path = Ex.staging.extract_default_config(package_location, temp_dir)

with open(def_conf_path, 'r') as fp:
    def_conf_dict = json.load(fp)

new_def_conf_dicts={}
list_dicts=[]
for video_data in Ex.val_data:
    new_def_conf_dict = def_conf_dict.copy()
    detects_path = video_data['video_path'].replace('processed','detections').replace('.mp4', '.json')
    new_def_conf_dict['detector_config'] = {"source_path": detects_path,"_class_name": "PassThroughDetectorConfig","batch_size": 1}
    new_def_conf_dict['video_config'] = video_data
    new_def_conf_dict['video_config']['_class_name'] = 'RawVideoConfig'
    list_dicts.append(new_def_conf_dict)

In [10]:
measurement_ids = ['M01-PS0304-001', 'M02-PP0214-000', 'M00-ChvUtd-000', 'M03-ER0429-001']
minimum_detections = [23,26,29,32,35,38,41,44]

In [ ]:
list_configs=[]
for min_det in minimum_detections:
    for config in list_dicts:
        current_config_dict = config.copy()
        flat_dict = flatten_dict(current_config_dict)
        updated_config_dict = expand_dict(flat_dict)
        updated_config_dict["filter_configs"] = [{"_class_name": 'BbAreaFilterConfig', 'min_detections': min_det, "max_pixel_area": 5000, "min_pixel_area": 100}]
        file_name = os.path.join(temp_dir, "M{:02d}-{}-{:02d}.json".format(len(list_configs), 
                                            current_config_dict['video_config']['video_id'], min_det))
        list_configs.append(file_name)
        with open(file_name, 'w') as f:
            json.dump(updated_config_dict, f, indent=2)
print(len(list_configs), list_configs[0])

In [ ]:
experiment = Ex.AWSExperiment(
    experiment_id="PTA-95-DET-1-4", job_queue=Ex.AWSComputeQueue.cpu_queue
)

for i, config_path in enumerate(list_configs):
    meas_id = os.path.basename(config_path).replace(".json", "")
    experiment.add_measurement(
        Ex.AWSMeasurement(
            experiment_id = "PTA-95-DET-1-4",
            measurement_id = meas_id,
            job_queue = Ex.AWSComputeQueue.cpu_queue,
            package_path = package_location,
            config_path = config_path
        )
    )

In [13]:
import boto3
experiment._batch_client = boto3.client('batch', region_name='us-east-1')

In [ ]:
experiment.stage_experiment()

In [3]:
experiment.launch_experiment()

In [3]:
import pandas as pd

In [4]:
data = pd.read_csv('/Users/jason.katz/Downloads/DET4.csv')

In [5]:
from bokeh.charts import Scatter
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row

In [6]:
output_notebook()

Loading BokehJS ...

In [ ]:
data['min_det'] = 1
for index, row in data.iterrows():
    data['min_det'][index] = row['job_id'][-3:]
data['min_det'][data['min_det']<0]=data['min_det'][data['min_det']<0]*-1

In [ ]:
data['game'] = 'id'
for index, row in data.iterrows():
    data['game'][index] = row['job_id'][19:25]

In [9]:
scat1 = Scatter(data, x='min_det', y='mota', color='game')
show(scat1)

In [10]:
data_min_det = data.copy()
data_min_det['minimum_detections'] = data_min_det['min_det']
data_min_det = data_min_det.groupby(['min_det']).mean()
data_min_det

,mota,motp,motr,mostly_tracked,mt_auc,minimum_detections
min_det,,,,,,
23,0.768034,0.636297,0.889955,0.808788,0.877075,23
26,0.768404,0.636432,0.888687,0.808449,0.875934,26
29,0.768634,0.636534,0.887212,0.806059,0.874579,29
32,0.768738,0.636723,0.885541,0.803976,0.873068,32
35,0.768593,0.636774,0.884096,0.798583,0.871726,35
38,0.768514,0.636904,0.882170,0.796493,0.869966,38
41,0.768271,0.637092,0.880319,0.794691,0.868268,41
44,0.767750,0.637239,0.878373,0.790854,0.866420,44


In [11]:
scat2 = Scatter(data_min_det, x = 'minimum_detections', y='mota')
scat3 = Scatter(data_min_det, x = 'minimum_detections', y='motp')
scat4 = Scatter(data_min_det, x = 'minimum_detections', y='mostly_tracked')
scat5 = Scatter(data_min_det, x = 'minimum_detections', y='mt_auc')
from bokeh.layouts import column, row
show(column(row(scat2, scat3), row(scat4, scat5)))